# 중고시장의 리셀제품 분석 프로젝트(가제)

## 합리적인 나이키 신발 구매

### 분석목적
- 가격 변화
- 중고가 ~ 현물가(머스트잇,크림) 비교
- 같은 제품의 플랫폼별(중고나라, 번개장터) 가격 비교 (강사님께 질문)
- 기간별 중고로 풀리는 명품 or 중고 수량 비교 (많이 풀리거나 떨어지는 기간이 있다면 인사이트가 될 것으로 기대)
- 네이버 검색량과 비교
- 허탕 비율 (내 검색어로 검색했는데 다른 상품이나왔을때) - 만약 이게 높다면 타사에 비해 경쟁력이 떨어지는 원인으로 인식될 수 있음
- 중고나라의 시세 추이가 현물 시세랑 얼마나 차이나는지/비슷한지
- 재발매일, 출시일에 대한 정보도 추가

### 데이터
- 날짜(일자별 데이터 from 중고나라)
- 등록 수 (from 중고나라)
- 가격 (중고나라 가격, 크림 가격-리셀, 정가)
- 상태(양호/보통/불량)
- 신발유형 (M/GS/PS)
- 검색어 트렌드 (from 네이버 데이터랩)

# 번개장터 데이터 설명

from : https://m.bunjang.co.kr/  (중고상품 판매 플랫폼 번개장터)

### 컬럼 설명

**굵은 글씨**는 우리 분석에 꼭 필요한 컬럼

- location : 등록 주소
- **name** : 판매자가 등록한 상품명
- **num_faved** : 좋아요 수
- pid : 상품아이디, @@@@@@@에 입력하면 해당 페이지로 이동함.-> https://m.bunjang.co.kr/products/@@@@@@@?q=%EB%82%98%EC%9D%B4%ED%82%A4%20%EB%8D%A9%ED%81%AC%EB%A1%9C%EC%9A%B0%20%EB%B2%94%EA%B3%A0%EB%9E%98&ref=%EA%B2%80%EC%83%89%EA%B2%B0%EA%B3%BC
- **price** : 가격
- product_image : 이미지 주소
- tag : 태그
- uid : 상품자 아이디? 아직 잘 모름
- status (0 = 판매중 / 1 = 예약완료 / 3 = 판매완료) : 상품 판매 상태
- **update_time** : 등록일 (<font color=red>기준은 찾아서 datetime형식으로 변환해야함</font>)
- **used** (1 = 중고 / 2 = 새상품 / 13 = 중고) : 상품상태  <font color=red>1과 13의 차이 아직 찾지 못했음</font>

In [310]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime, timedelta

pd.options.display.max_rows=100
pd.options.display.max_columns=100

%matplotlib inline

from matplotlib import font_manager, rc
f_path = "c:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=f_path).get_name()
rc('font', family=font_name)

# Load Data

In [342]:
df = pd.read_csv('./data/NIKE_Jordan1Low.csv')
df.head() # 최신순

,Unnamed: 0,ad,bizseller,checkout,contact_hope,free_shipping,is_adult,is_super_up_shop,location,max_cpc,name,num_comment,num_faved,only_neighborhood,outlink_url,pid,price,product_image,pu_id,ref_campaign,ref_code,ref_medium,ref_content,ref_source,status,style,super_up,tag,uid,update_time,used,bun_pay_filter_enabled,imp_id,ad_ref,faved,datetime,year,month,day,week,weekday,hour
0,0,False,False,False,False,False,False,NaN,부산광역시 해운대구 재송제1동,NaN,조던1 로우 그린토 260,0,7,False,NaN,167630285,200000,https://media.bunjang.co.kr/product/167630285_...,NaN,NaN,soldout_test_v2:B,NaN,NaN,NaN,0,NaN,NaN,나이키 조던 조던1 조던1로우 그린토,12507591,1636597819,2,True,3095618c80ca032b33d7,NaN,False,2021-11-11 11:30:19,2021,11,11,45,3,11
1,1,False,False,False,False,True,False,NaN,대구광역시 달서구 진천동,NaN,트래비스스캇 프라그먼트 조던1 로우 245,0,8,False,NaN,168674338,1110000,https://media.bunjang.co.kr/product/168674338_...,NaN,NaN,soldout_test_v2:B,NaN,NaN,NaN,0,NaN,NaN,나이키트래비스캇프라그먼트 스캇프라그먼트 조던1스캇프라그먼트 스캇조던1 프라그먼트스캇로우,7673058,1636597645,1,False,3095618c80ca032b33d8,NaN,False,2021-11-11 11:27:25,2021,11,11,45,3,11
2,2,False,False,False,False,True,False,NaN,NaN,NaN,조던1로우 스타피쉬 W 245,0,4,False,NaN,169268036,250000,https://media.bunjang.co.kr/product/169268036_...,NaN,NaN,soldout_test_v2:B,NaN,NaN,NaN,0,NaN,NaN,덩크로우 나이키덩크로우범고래 나이키덩크로우 스타피쉬 조던1,195093,1636597642,2,False,3095618c80ca032b33d9,NaN,False,2021-11-11 11:27:22,2021,11,11,45,3,11
3,3,False,False,False,False,False,False,NaN,경상북도 칠곡군 북삼읍,NaN,조던1로우 파인그린,0,7,False,NaN,168265090,110000,https://media.bunjang.co.kr/product/168265090_...,NaN,NaN,soldout_test_v2:B,NaN,NaN,NaN,0,NaN,NaN,조던1 조던1270 로우 조던1파인그린 조던파인그린,7037926,1636597581,1,True,3095618c80ca032b33da,NaN,False,2021-11-11 11:26:21,2021,11,11,45,3,11
4,4,False,False,False,False,True,False,NaN,서울특별시 강동구 성내제2동,NaN,[250] 에어 조던 11로우 스네이크 라이트본,0,0,False,NaN,169579769,150000,https://media.bunjang.co.kr/product/169579769_...,NaN,NaN,soldout_test_v2:B,NaN,NaN,NaN,0,NaN,NaN,나이키 조던 뱀피 조던11 조던로우,3331253,1636597576,1,True,3095618c80ca032b33db,NaN,False,2021-11-11 11:26:16,2021,11,11,45,3,11


In [343]:
df1 = df.drop(df.columns[0], axis=1)
df = df1
df.head()

,ad,bizseller,checkout,contact_hope,free_shipping,is_adult,is_super_up_shop,location,max_cpc,name,num_comment,num_faved,only_neighborhood,outlink_url,pid,price,product_image,pu_id,ref_campaign,ref_code,ref_medium,ref_content,ref_source,status,style,super_up,tag,uid,update_time,used,bun_pay_filter_enabled,imp_id,ad_ref,faved,datetime,year,month,day,week,weekday,hour
0,False,False,False,False,False,False,NaN,부산광역시 해운대구 재송제1동,NaN,조던1 로우 그린토 260,0,7,False,NaN,167630285,200000,https://media.bunjang.co.kr/product/167630285_...,NaN,NaN,soldout_test_v2:B,NaN,NaN,NaN,0,NaN,NaN,나이키 조던 조던1 조던1로우 그린토,12507591,1636597819,2,True,3095618c80ca032b33d7,NaN,False,2021-11-11 11:30:19,2021,11,11,45,3,11
1,False,False,False,False,True,False,NaN,대구광역시 달서구 진천동,NaN,트래비스스캇 프라그먼트 조던1 로우 245,0,8,False,NaN,168674338,1110000,https://media.bunjang.co.kr/product/168674338_...,NaN,NaN,soldout_test_v2:B,NaN,NaN,NaN,0,NaN,NaN,나이키트래비스캇프라그먼트 스캇프라그먼트 조던1스캇프라그먼트 스캇조던1 프라그먼트스캇로우,7673058,1636597645,1,False,3095618c80ca032b33d8,NaN,False,2021-11-11 11:27:25,2021,11,11,45,3,11
2,False,False,False,False,True,False,NaN,NaN,NaN,조던1로우 스타피쉬 W 245,0,4,False,NaN,169268036,250000,https://media.bunjang.co.kr/product/169268036_...,NaN,NaN,soldout_test_v2:B,NaN,NaN,NaN,0,NaN,NaN,덩크로우 나이키덩크로우범고래 나이키덩크로우 스타피쉬 조던1,195093,1636597642,2,False,3095618c80ca032b33d9,NaN,False,2021-11-11 11:27:22,2021,11,11,45,3,11
3,False,False,False,False,False,False,NaN,경상북도 칠곡군 북삼읍,NaN,조던1로우 파인그린,0,7,False,NaN,168265090,110000,https://media.bunjang.co.kr/product/168265090_...,NaN,NaN,soldout_test_v2:B,NaN,NaN,NaN,0,NaN,NaN,조던1 조던1270 로우 조던1파인그린 조던파인그린,7037926,1636597581,1,True,3095618c80ca032b33da,NaN,False,2021-11-11 11:26:21,2021,11,11,45,3,11
4,False,False,False,False,True,False,NaN,서울특별시 강동구 성내제2동,NaN,[250] 에어 조던 11로우 스네이크 라이트본,0,0,False,NaN,169579769,150000,https://media.bunjang.co.kr/product/169579769_...,NaN,NaN,soldout_test_v2:B,NaN,NaN,NaN,0,NaN,NaN,나이키 조던 뱀피 조던11 조던로우,3331253,1636597576,1,True,3095618c80ca032b33db,NaN,False,2021-11-11 11:26:16,2021,11,11,45,3,11


# Column 정리

In [344]:
columns = ["location", "name","num_faved", "pid", "price","status","uid","update_time","used"]
df_C = df.loc[: ,columns]
df_C

,location,name,num_faved,pid,price,status,uid,update_time,used
0,부산광역시 해운대구 재송제1동,조던1 로우 그린토 260,7,167630285,200000,0,12507591,1636597819,2
1,대구광역시 달서구 진천동,트래비스스캇 프라그먼트 조던1 로우 245,8,168674338,1110000,0,7673058,1636597645,1
2,NaN,조던1로우 스타피쉬 W 245,4,169268036,250000,0,195093,1636597642,2
3,경상북도 칠곡군 북삼읍,조던1로우 파인그린,7,168265090,110000,0,7037926,1636597581,1
4,서울특별시 강동구 성내제2동,[250] 에어 조던 11로우 스네이크 라이트본,0,169579769,150000,0,3331253,1636597576,1
...,...,...,...,...,...,...,...,...,...
4455,서울특별시 동대문구 장안제1동,조던 11 브레드 로우 팝니다,0,112221723,60000,3,2126591,1607427388,13
4456,경기도 안산시 상록구 본오3동,(250)조던11 로우 뱀피,7,137891040,100000,3,8659863,1605255376,13
4457,경기도 화성시 동탄3동,[230] 조던1 로우 se orange blaze 팝니다 - 새제품,2,128192751,139000,3,6017020,1595131995,13
4458,경기도 오산시 대원동,오시야/나이키 조던 로우/상태A+/사이즈(240)/S1290,3,125090987,45000,3,1861967,1590032854,1


# update time 전처리

In [345]:
# 파라미터를 기점으로 경과된 시간('update_time')을 연월일 시간으로 변환
def get_date(x):
    return datetime(1970,1,1,9) + timedelta(seconds=int(x))

def datetime_maker(df):
    df['datetime'] = df['update_time'].apply(get_date)
    return df

## 시간,분,초 삭제하고 새로운 date 컬럼 생성 
def dt_index(x):
    return str(x)[:10]

In [346]:
df_CT = df_C.copy()

df_CT = datetime_maker(df_C)
df_CT.head()

df_CT['date'] = df_CT['datetime'].apply(dt_index)
df_CT.head()

,location,name,num_faved,pid,price,status,uid,update_time,used,datetime,date
0,부산광역시 해운대구 재송제1동,조던1 로우 그린토 260,7,167630285,200000,0,12507591,1636597819,2,2021-11-11 11:30:19,2021-11-11
1,대구광역시 달서구 진천동,트래비스스캇 프라그먼트 조던1 로우 245,8,168674338,1110000,0,7673058,1636597645,1,2021-11-11 11:27:25,2021-11-11
2,NaN,조던1로우 스타피쉬 W 245,4,169268036,250000,0,195093,1636597642,2,2021-11-11 11:27:22,2021-11-11
3,경상북도 칠곡군 북삼읍,조던1로우 파인그린,7,168265090,110000,0,7037926,1636597581,1,2021-11-11 11:26:21,2021-11-11
4,서울특별시 강동구 성내제2동,[250] 에어 조던 11로우 스네이크 라이트본,0,169579769,150000,0,3331253,1636597576,1,2021-11-11 11:26:16,2021-11-11


# name 칼럼에서 size 추출

In [347]:
import re
size = []
# df index 범위만큼 for문 돌림 #
for i in range(0, len(df_CT)):
    size.append(re.sub(r'[^0-9]','',df_CT['name'].values[i]))

df_CT["size"] = pd.DataFrame(size)

In [348]:
df = df_CT
df

,location,name,num_faved,pid,price,status,uid,update_time,used,datetime,date,size
0,부산광역시 해운대구 재송제1동,조던1 로우 그린토 260,7,167630285,200000,0,12507591,1636597819,2,2021-11-11 11:30:19,2021-11-11,1260
1,대구광역시 달서구 진천동,트래비스스캇 프라그먼트 조던1 로우 245,8,168674338,1110000,0,7673058,1636597645,1,2021-11-11 11:27:25,2021-11-11,1245
2,NaN,조던1로우 스타피쉬 W 245,4,169268036,250000,0,195093,1636597642,2,2021-11-11 11:27:22,2021-11-11,1245
3,경상북도 칠곡군 북삼읍,조던1로우 파인그린,7,168265090,110000,0,7037926,1636597581,1,2021-11-11 11:26:21,2021-11-11,1
4,서울특별시 강동구 성내제2동,[250] 에어 조던 11로우 스네이크 라이트본,0,169579769,150000,0,3331253,1636597576,1,2021-11-11 11:26:16,2021-11-11,25011


## name == [제품명, 색깔] 데이터 추출

In [349]:
# 띄어쓰기에 따라서 [제품명, 색깔 등]를 포함하는 name으로 필터링
target_string = ['조던', '1', '로우', '울프', '그레이']
sample = df[df['name'].map(lambda x: all(string in x for string in target_string))]
df = sample
df = df.reset_index(drop=True)
df

In [350]:
# 2차 '제품명' raw 데이터 저장
df.to_csv('./data/jordan1_low_wolfgray.csv')

In [351]:
# '우먼스' 제품 제거
idx_wm = df[df['name'].str.contains('우먼스')].index 
sample = df.drop(idx_wm)

df = sample
df = df.reset_index(drop=True)
df

In [352]:
# "삽니다", "구매" 구매요청 게시글 제거
idx_buying = df[df['name'].str.contains('삽니다|구매')].index
sample = df.drop(idx_buying)
df = sample
df = df.reset_index(drop=True)
df

,location,name,num_faved,pid,price,status,uid,update_time,used,datetime,date,size
0,서울특별시 송파구 풍납2동,[240] 나이키 조던 1 로우 화이트 울프 그레이,3,169417451,320000,0,2652234,1636597127,2,2021-11-11 11:18:47,2021-11-11,2401
1,경기도 수원시 영통구 태장동,조던1 로우 울프그레이 230,9,169066756,268000,0,77817142,1636592808,2,2021-11-11 10:06:48,2021-11-11,1230
2,경상북도 경산시 진량읍,나이키 조던1 로우 울프그레이 (270),7,169404484,300000,1,4897938,1636589159,1,2021-11-11 09:05:59,2021-11-11,1270
3,서울특별시 강서구 화곡제1동,나이키 조던1 로우 울프그레이245,7,169448963,230000,0,2199505,1636582425,1,2021-11-11 07:13:45,2021-11-11,1245
4,서울특별시 영등포구 문래동,조던 1 로우 화이트 울프 그레이 265,1,168304657,380000,0,77568690,1636577518,2,2021-11-11 05:51:58,2021-11-11,1265
...,...,...,...,...,...,...,...,...,...,...,...,...
204,경기도 남양주시 오남읍,조던1 로우 울프그레이 240,1,161646308,240000,3,1278407,1629010574,1,2021-08-15 15:56:14,2021-08-15,1240
205,인천광역시 서구 청라2동,(w)조던 1 로우 화이트 울프 그레이,2,161156153,290000,3,8455531,1629000408,1,2021-08-15 13:06:48,2021-08-15,1
206,경기도 남양주시 다산1동,[235]조던1 로우 울프그레이,2,161695127,225000,3,10281499,1628938834,1,2021-08-14 20:00:34,2021-08-14,2351
207,부산광역시 수영구 민락동,나이키 조던 로우1 울프그레이,5,161628686,265000,3,2910383,1628899012,1,2021-08-14 08:56:52,2021-08-14,1


# product_name, brand, product_type, color 칼럼 추가

In [353]:
df['product_name'] = '조던 1 로우 울프그레이'
df['brand'] = '조던'
df['product_type'] = '조던 1 로우'
df['color'] = '울프그레이'

In [354]:
df = df.reset_index(drop=True)
df

,location,name,num_faved,pid,price,status,uid,update_time,used,datetime,date,size,product_name,brand,product_type,color
0,서울특별시 송파구 풍납2동,[240] 나이키 조던 1 로우 화이트 울프 그레이,3,169417451,320000,0,2652234,1636597127,2,2021-11-11 11:18:47,2021-11-11,2401,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
1,경기도 수원시 영통구 태장동,조던1 로우 울프그레이 230,9,169066756,268000,0,77817142,1636592808,2,2021-11-11 10:06:48,2021-11-11,1230,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
2,경상북도 경산시 진량읍,나이키 조던1 로우 울프그레이 (270),7,169404484,300000,1,4897938,1636589159,1,2021-11-11 09:05:59,2021-11-11,1270,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
3,서울특별시 강서구 화곡제1동,나이키 조던1 로우 울프그레이245,7,169448963,230000,0,2199505,1636582425,1,2021-11-11 07:13:45,2021-11-11,1245,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
4,서울특별시 영등포구 문래동,조던 1 로우 화이트 울프 그레이 265,1,168304657,380000,0,77568690,1636577518,2,2021-11-11 05:51:58,2021-11-11,1265,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,경기도 남양주시 오남읍,조던1 로우 울프그레이 240,1,161646308,240000,3,1278407,1629010574,1,2021-08-15 15:56:14,2021-08-15,1240,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
205,인천광역시 서구 청라2동,(w)조던 1 로우 화이트 울프 그레이,2,161156153,290000,3,8455531,1629000408,1,2021-08-15 13:06:48,2021-08-15,1,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
206,경기도 남양주시 다산1동,[235]조던1 로우 울프그레이,2,161695127,225000,3,10281499,1628938834,1,2021-08-14 20:00:34,2021-08-14,2351,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
207,부산광역시 수영구 민락동,나이키 조던 로우1 울프그레이,5,161628686,265000,3,2910383,1628899012,1,2021-08-14 08:56:52,2021-08-14,1,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이


# size 처리

In [355]:
import re

In [356]:
df[df['size'] == '1']

,location,name,num_faved,pid,price,status,uid,update_time,used,datetime,date,size,product_name,brand,product_type,color
18,전라북도 전주시 덕진구 인후3동,조던1로우 울프그레이,14,164753786,400000,0,1768555,1636427574,2,2021-11-09 12:12:54,2021-11-09,1,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
27,NaN,(W) 조던 1 로우 화이트 울프 그레이,9,167076609,349000,0,77657935,1636167707,2,2021-11-06 12:01:47,2021-11-06,1,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
29,NaN,조던1 로우 화이트 울프 그레이,1,168902710,260000,0,2214577,1635983170,1,2021-11-04 08:46:10,2021-11-04,1,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
30,NaN,조던1 로우 화이트 울프 그레이,0,168902624,260000,0,2214577,1635983048,1,2021-11-04 08:44:08,2021-11-04,1,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
31,충청북도 충주시 목행.용탄동,조던1로우 화이트 울프그레이,4,166300228,310000,0,545127,1635910889,1,2021-11-03 12:41:29,2021-11-03,1,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
34,강원도 춘천시 퇴계동,나이키 조던1 로우 울프 그레이,10,166187958,300000,0,818779,1635415226,1,2021-10-28 19:00:26,2021-10-28,1,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
39,NaN,w 조던1 로우 화이트 울프 그레이,1,165493435,333000,0,6731072,1633239707,1,2021-10-03 14:41:47,2021-10-03,1,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
51,부산광역시 동래구 명륜동,조던 1로우 울프그레이,1,168951446,253000,3,4385835,1636380881,1,2021-11-08 23:14:41,2021-11-08,1,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
52,대전광역시 중구 오류동,나이키 조던1 로우 화이트 울프 그레이,6,168696427,270000,3,74855337,1636326935,1,2021-11-08 08:15:35,2021-11-08,1,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
58,부산광역시 부산진구 부전제1동,조던 1 로우 울프그레이,1,168886729,230000,3,6620269,1635950346,1,2021-11-03 23:39:06,2021-11-03,1,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이


In [357]:
# 사이즈 없는 size == '1' 값 '0' 처리
sample = df.copy()
sample['size'] = df['size'].replace('1', '0')
sample[sample['size'] == '0']

df = sample
df

,location,name,num_faved,pid,price,status,uid,update_time,used,datetime,date,size,product_name,brand,product_type,color
0,서울특별시 송파구 풍납2동,[240] 나이키 조던 1 로우 화이트 울프 그레이,3,169417451,320000,0,2652234,1636597127,2,2021-11-11 11:18:47,2021-11-11,2401,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
1,경기도 수원시 영통구 태장동,조던1 로우 울프그레이 230,9,169066756,268000,0,77817142,1636592808,2,2021-11-11 10:06:48,2021-11-11,1230,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
2,경상북도 경산시 진량읍,나이키 조던1 로우 울프그레이 (270),7,169404484,300000,1,4897938,1636589159,1,2021-11-11 09:05:59,2021-11-11,1270,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
3,서울특별시 강서구 화곡제1동,나이키 조던1 로우 울프그레이245,7,169448963,230000,0,2199505,1636582425,1,2021-11-11 07:13:45,2021-11-11,1245,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
4,서울특별시 영등포구 문래동,조던 1 로우 화이트 울프 그레이 265,1,168304657,380000,0,77568690,1636577518,2,2021-11-11 05:51:58,2021-11-11,1265,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,경기도 남양주시 오남읍,조던1 로우 울프그레이 240,1,161646308,240000,3,1278407,1629010574,1,2021-08-15 15:56:14,2021-08-15,1240,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
205,인천광역시 서구 청라2동,(w)조던 1 로우 화이트 울프 그레이,2,161156153,290000,3,8455531,1629000408,1,2021-08-15 13:06:48,2021-08-15,0,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
206,경기도 남양주시 다산1동,[235]조던1 로우 울프그레이,2,161695127,225000,3,10281499,1628938834,1,2021-08-14 20:00:34,2021-08-14,2351,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
207,부산광역시 수영구 민락동,나이키 조던 로우1 울프그레이,5,161628686,265000,3,2910383,1628899012,1,2021-08-14 08:56:52,2021-08-14,0,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이


In [358]:
# size 앞뒤로 붙어있는 '1' 값 제거

# def strip_one(x):
#     return x.strip('1')
# product['size'].apply(strip_one)

sample = df.copy()
sample1 = []

for i in range(0, len(df)):
    sample1.append(sample['size'][i].strip('1'))

set(sample1)

sample["size"] = pd.DataFrame(sample1)
sample['size']

df = sample
df.reset_index(drop=True)
df

,location,name,num_faved,pid,price,status,uid,update_time,used,datetime,date,size,product_name,brand,product_type,color
0,서울특별시 송파구 풍납2동,[240] 나이키 조던 1 로우 화이트 울프 그레이,3,169417451,320000,0,2652234,1636597127,2,2021-11-11 11:18:47,2021-11-11,240,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
1,경기도 수원시 영통구 태장동,조던1 로우 울프그레이 230,9,169066756,268000,0,77817142,1636592808,2,2021-11-11 10:06:48,2021-11-11,230,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
2,경상북도 경산시 진량읍,나이키 조던1 로우 울프그레이 (270),7,169404484,300000,1,4897938,1636589159,1,2021-11-11 09:05:59,2021-11-11,270,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
3,서울특별시 강서구 화곡제1동,나이키 조던1 로우 울프그레이245,7,169448963,230000,0,2199505,1636582425,1,2021-11-11 07:13:45,2021-11-11,245,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
4,서울특별시 영등포구 문래동,조던 1 로우 화이트 울프 그레이 265,1,168304657,380000,0,77568690,1636577518,2,2021-11-11 05:51:58,2021-11-11,265,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,경기도 남양주시 오남읍,조던1 로우 울프그레이 240,1,161646308,240000,3,1278407,1629010574,1,2021-08-15 15:56:14,2021-08-15,240,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
205,인천광역시 서구 청라2동,(w)조던 1 로우 화이트 울프 그레이,2,161156153,290000,3,8455531,1629000408,1,2021-08-15 13:06:48,2021-08-15,0,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
206,경기도 남양주시 다산1동,[235]조던1 로우 울프그레이,2,161695127,225000,3,10281499,1628938834,1,2021-08-14 20:00:34,2021-08-14,235,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
207,부산광역시 수영구 민락동,나이키 조던 로우1 울프그레이,5,161628686,265000,3,2910383,1628899012,1,2021-08-14 08:56:52,2021-08-14,0,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이


In [359]:
## size 3자리 수 이상의 이상치 제거
sample = df.copy()

len_size = []

for i in range(0, len(df)):
    len_size.append(len(df['size'].values[i]))
    
sample["len_size"] = pd.DataFrame(len_size)
set(sample["len_size"].values)


idx = sample[sample['len_size'] > 3].index
sample = sample.drop(idx)
sample

df = sample
df = df.drop(['len_size'], axis=1)
df

,location,name,num_faved,pid,price,status,uid,update_time,used,datetime,date,size,product_name,brand,product_type,color
0,서울특별시 송파구 풍납2동,[240] 나이키 조던 1 로우 화이트 울프 그레이,3,169417451,320000,0,2652234,1636597127,2,2021-11-11 11:18:47,2021-11-11,240,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
1,경기도 수원시 영통구 태장동,조던1 로우 울프그레이 230,9,169066756,268000,0,77817142,1636592808,2,2021-11-11 10:06:48,2021-11-11,230,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
2,경상북도 경산시 진량읍,나이키 조던1 로우 울프그레이 (270),7,169404484,300000,1,4897938,1636589159,1,2021-11-11 09:05:59,2021-11-11,270,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
3,서울특별시 강서구 화곡제1동,나이키 조던1 로우 울프그레이245,7,169448963,230000,0,2199505,1636582425,1,2021-11-11 07:13:45,2021-11-11,245,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
4,서울특별시 영등포구 문래동,조던 1 로우 화이트 울프 그레이 265,1,168304657,380000,0,77568690,1636577518,2,2021-11-11 05:51:58,2021-11-11,265,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,경기도 남양주시 오남읍,조던1 로우 울프그레이 240,1,161646308,240000,3,1278407,1629010574,1,2021-08-15 15:56:14,2021-08-15,240,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
205,인천광역시 서구 청라2동,(w)조던 1 로우 화이트 울프 그레이,2,161156153,290000,3,8455531,1629000408,1,2021-08-15 13:06:48,2021-08-15,0,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
206,경기도 남양주시 다산1동,[235]조던1 로우 울프그레이,2,161695127,225000,3,10281499,1628938834,1,2021-08-14 20:00:34,2021-08-14,235,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
207,부산광역시 수영구 민락동,나이키 조던 로우1 울프그레이,5,161628686,265000,3,2910383,1628899012,1,2021-08-14 08:56:52,2021-08-14,0,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이


In [360]:
### size.dtype -> int로 변환
df['size'] = df['size'].astype(int)
df['size']

df = df.reset_index(drop=True)
df

,location,name,num_faved,pid,price,status,uid,update_time,used,datetime,date,size,product_name,brand,product_type,color
0,서울특별시 송파구 풍납2동,[240] 나이키 조던 1 로우 화이트 울프 그레이,3,169417451,320000,0,2652234,1636597127,2,2021-11-11 11:18:47,2021-11-11,240,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
1,경기도 수원시 영통구 태장동,조던1 로우 울프그레이 230,9,169066756,268000,0,77817142,1636592808,2,2021-11-11 10:06:48,2021-11-11,230,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
2,경상북도 경산시 진량읍,나이키 조던1 로우 울프그레이 (270),7,169404484,300000,1,4897938,1636589159,1,2021-11-11 09:05:59,2021-11-11,270,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
3,서울특별시 강서구 화곡제1동,나이키 조던1 로우 울프그레이245,7,169448963,230000,0,2199505,1636582425,1,2021-11-11 07:13:45,2021-11-11,245,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
4,서울특별시 영등포구 문래동,조던 1 로우 화이트 울프 그레이 265,1,168304657,380000,0,77568690,1636577518,2,2021-11-11 05:51:58,2021-11-11,265,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,경기도 남양주시 오남읍,조던1 로우 울프그레이 240,1,161646308,240000,3,1278407,1629010574,1,2021-08-15 15:56:14,2021-08-15,240,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
182,인천광역시 서구 청라2동,(w)조던 1 로우 화이트 울프 그레이,2,161156153,290000,3,8455531,1629000408,1,2021-08-15 13:06:48,2021-08-15,0,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
183,경기도 남양주시 다산1동,[235]조던1 로우 울프그레이,2,161695127,225000,3,10281499,1628938834,1,2021-08-14 20:00:34,2021-08-14,235,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
184,부산광역시 수영구 민락동,나이키 조던 로우1 울프그레이,5,161628686,265000,3,2910383,1628899012,1,2021-08-14 08:56:52,2021-08-14,0,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이


In [361]:
#### (size >= 200) & (size <= 350)
df = df[(df['size'] >= 200) & (df['size'] <= 350)]
df = df.reset_index(drop=True)
df

,location,name,num_faved,pid,price,status,uid,update_time,used,datetime,date,size,product_name,brand,product_type,color
0,서울특별시 송파구 풍납2동,[240] 나이키 조던 1 로우 화이트 울프 그레이,3,169417451,320000,0,2652234,1636597127,2,2021-11-11 11:18:47,2021-11-11,240,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
1,경기도 수원시 영통구 태장동,조던1 로우 울프그레이 230,9,169066756,268000,0,77817142,1636592808,2,2021-11-11 10:06:48,2021-11-11,230,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
2,경상북도 경산시 진량읍,나이키 조던1 로우 울프그레이 (270),7,169404484,300000,1,4897938,1636589159,1,2021-11-11 09:05:59,2021-11-11,270,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
3,서울특별시 강서구 화곡제1동,나이키 조던1 로우 울프그레이245,7,169448963,230000,0,2199505,1636582425,1,2021-11-11 07:13:45,2021-11-11,245,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
4,서울특별시 영등포구 문래동,조던 1 로우 화이트 울프 그레이 265,1,168304657,380000,0,77568690,1636577518,2,2021-11-11 05:51:58,2021-11-11,265,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,NaN,조던 1 로우 화이트 울프그레이 250,0,162000681,230000,3,77003151,1629189188,2,2021-08-17 17:33:08,2021-08-17,250,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
147,충청북도 청주시 상당구 용암2동,(새상품)조던1로우 울프그레이260,0,161834566,250000,3,5187880,1629022144,1,2021-08-15 19:09:04,2021-08-15,260,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
148,경기도 남양주시 오남읍,조던1 로우 울프그레이 240,1,161646308,240000,3,1278407,1629010574,1,2021-08-15 15:56:14,2021-08-15,240,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이
149,경기도 남양주시 다산1동,[235]조던1 로우 울프그레이,2,161695127,225000,3,10281499,1628938834,1,2021-08-14 20:00:34,2021-08-14,235,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이


# price 칼럼 처리
- '조던1 로우 스타피쉬' 정발가 : 159,000원
- '조던1 로우 트레비스스캇' 정발가 : 189,000원
- '조던1 로우 울프그레이' 정발가 : 119,000원

In [362]:
## 정발가 칼럼 추가
df['release_price'] = 119000

In [363]:
df = df.reset_index(drop=True)
df

,location,name,num_faved,pid,price,status,uid,update_time,used,datetime,date,size,product_name,brand,product_type,color,release_price
0,서울특별시 송파구 풍납2동,[240] 나이키 조던 1 로우 화이트 울프 그레이,3,169417451,320000,0,2652234,1636597127,2,2021-11-11 11:18:47,2021-11-11,240,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000
1,경기도 수원시 영통구 태장동,조던1 로우 울프그레이 230,9,169066756,268000,0,77817142,1636592808,2,2021-11-11 10:06:48,2021-11-11,230,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000
2,경상북도 경산시 진량읍,나이키 조던1 로우 울프그레이 (270),7,169404484,300000,1,4897938,1636589159,1,2021-11-11 09:05:59,2021-11-11,270,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000
3,서울특별시 강서구 화곡제1동,나이키 조던1 로우 울프그레이245,7,169448963,230000,0,2199505,1636582425,1,2021-11-11 07:13:45,2021-11-11,245,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000
4,서울특별시 영등포구 문래동,조던 1 로우 화이트 울프 그레이 265,1,168304657,380000,0,77568690,1636577518,2,2021-11-11 05:51:58,2021-11-11,265,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,NaN,조던 1 로우 화이트 울프그레이 250,0,162000681,230000,3,77003151,1629189188,2,2021-08-17 17:33:08,2021-08-17,250,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000
147,충청북도 청주시 상당구 용암2동,(새상품)조던1로우 울프그레이260,0,161834566,250000,3,5187880,1629022144,1,2021-08-15 19:09:04,2021-08-15,260,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000
148,경기도 남양주시 오남읍,조던1 로우 울프그레이 240,1,161646308,240000,3,1278407,1629010574,1,2021-08-15 15:56:14,2021-08-15,240,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000
149,경기도 남양주시 다산1동,[235]조던1 로우 울프그레이,2,161695127,225000,3,10281499,1628938834,1,2021-08-14 20:00:34,2021-08-14,235,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000


# location 칼럼

In [364]:
df['location'] = df['location'].fillna('NULL')
df

,location,name,num_faved,pid,price,status,uid,update_time,used,datetime,date,size,product_name,brand,product_type,color,release_price
0,서울특별시 송파구 풍납2동,[240] 나이키 조던 1 로우 화이트 울프 그레이,3,169417451,320000,0,2652234,1636597127,2,2021-11-11 11:18:47,2021-11-11,240,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000
1,경기도 수원시 영통구 태장동,조던1 로우 울프그레이 230,9,169066756,268000,0,77817142,1636592808,2,2021-11-11 10:06:48,2021-11-11,230,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000
2,경상북도 경산시 진량읍,나이키 조던1 로우 울프그레이 (270),7,169404484,300000,1,4897938,1636589159,1,2021-11-11 09:05:59,2021-11-11,270,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000
3,서울특별시 강서구 화곡제1동,나이키 조던1 로우 울프그레이245,7,169448963,230000,0,2199505,1636582425,1,2021-11-11 07:13:45,2021-11-11,245,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000
4,서울특별시 영등포구 문래동,조던 1 로우 화이트 울프 그레이 265,1,168304657,380000,0,77568690,1636577518,2,2021-11-11 05:51:58,2021-11-11,265,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,NULL,조던 1 로우 화이트 울프그레이 250,0,162000681,230000,3,77003151,1629189188,2,2021-08-17 17:33:08,2021-08-17,250,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000
147,충청북도 청주시 상당구 용암2동,(새상품)조던1로우 울프그레이260,0,161834566,250000,3,5187880,1629022144,1,2021-08-15 19:09:04,2021-08-15,260,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000
148,경기도 남양주시 오남읍,조던1 로우 울프그레이 240,1,161646308,240000,3,1278407,1629010574,1,2021-08-15 15:56:14,2021-08-15,240,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000
149,경기도 남양주시 다산1동,[235]조던1 로우 울프그레이,2,161695127,225000,3,10281499,1628938834,1,2021-08-14 20:00:34,2021-08-14,235,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000


In [365]:
def first_split(x):
    first = x.find(' ')
    return x[:first]

def second_split(x):
    first = x.find(' ')
    second = x.find(' ', first+1)
    return x[first+1:second]

def third_split(x):
    first = x.find(' ')
    second = x.find(' ', first+1)
    third = x.find(' ', second+1)
    return x[second+1:]

In [366]:
df['location1'] = df['location'].apply(first_split)
df['location2'] = df['location'].apply(second_split)
df['location3'] = df['location'].apply(third_split)
df

,location,name,num_faved,pid,price,status,uid,update_time,used,datetime,date,size,product_name,brand,product_type,color,release_price,location1,location2,location3
0,서울특별시 송파구 풍납2동,[240] 나이키 조던 1 로우 화이트 울프 그레이,3,169417451,320000,0,2652234,1636597127,2,2021-11-11 11:18:47,2021-11-11,240,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000,서울특별시,송파구,풍납2동
1,경기도 수원시 영통구 태장동,조던1 로우 울프그레이 230,9,169066756,268000,0,77817142,1636592808,2,2021-11-11 10:06:48,2021-11-11,230,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000,경기도,수원시,영통구 태장동
2,경상북도 경산시 진량읍,나이키 조던1 로우 울프그레이 (270),7,169404484,300000,1,4897938,1636589159,1,2021-11-11 09:05:59,2021-11-11,270,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000,경상북도,경산시,진량읍
3,서울특별시 강서구 화곡제1동,나이키 조던1 로우 울프그레이245,7,169448963,230000,0,2199505,1636582425,1,2021-11-11 07:13:45,2021-11-11,245,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000,서울특별시,강서구,화곡제1동
4,서울특별시 영등포구 문래동,조던 1 로우 화이트 울프 그레이 265,1,168304657,380000,0,77568690,1636577518,2,2021-11-11 05:51:58,2021-11-11,265,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000,서울특별시,영등포구,문래동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,NULL,조던 1 로우 화이트 울프그레이 250,0,162000681,230000,3,77003151,1629189188,2,2021-08-17 17:33:08,2021-08-17,250,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000,NUL,NUL,NULL
147,충청북도 청주시 상당구 용암2동,(새상품)조던1로우 울프그레이260,0,161834566,250000,3,5187880,1629022144,1,2021-08-15 19:09:04,2021-08-15,260,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000,충청북도,청주시,상당구 용암2동
148,경기도 남양주시 오남읍,조던1 로우 울프그레이 240,1,161646308,240000,3,1278407,1629010574,1,2021-08-15 15:56:14,2021-08-15,240,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000,경기도,남양주시,오남읍
149,경기도 남양주시 다산1동,[235]조던1 로우 울프그레이,2,161695127,225000,3,10281499,1628938834,1,2021-08-14 20:00:34,2021-08-14,235,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000,경기도,남양주시,다산1동


# 전처리 완료! .csv로 저장

In [367]:
df

,location,name,num_faved,pid,price,status,uid,update_time,used,datetime,date,size,product_name,brand,product_type,color,release_price,location1,location2,location3
0,서울특별시 송파구 풍납2동,[240] 나이키 조던 1 로우 화이트 울프 그레이,3,169417451,320000,0,2652234,1636597127,2,2021-11-11 11:18:47,2021-11-11,240,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000,서울특별시,송파구,풍납2동
1,경기도 수원시 영통구 태장동,조던1 로우 울프그레이 230,9,169066756,268000,0,77817142,1636592808,2,2021-11-11 10:06:48,2021-11-11,230,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000,경기도,수원시,영통구 태장동
2,경상북도 경산시 진량읍,나이키 조던1 로우 울프그레이 (270),7,169404484,300000,1,4897938,1636589159,1,2021-11-11 09:05:59,2021-11-11,270,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000,경상북도,경산시,진량읍
3,서울특별시 강서구 화곡제1동,나이키 조던1 로우 울프그레이245,7,169448963,230000,0,2199505,1636582425,1,2021-11-11 07:13:45,2021-11-11,245,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000,서울특별시,강서구,화곡제1동
4,서울특별시 영등포구 문래동,조던 1 로우 화이트 울프 그레이 265,1,168304657,380000,0,77568690,1636577518,2,2021-11-11 05:51:58,2021-11-11,265,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000,서울특별시,영등포구,문래동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,NULL,조던 1 로우 화이트 울프그레이 250,0,162000681,230000,3,77003151,1629189188,2,2021-08-17 17:33:08,2021-08-17,250,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000,NUL,NUL,NULL
147,충청북도 청주시 상당구 용암2동,(새상품)조던1로우 울프그레이260,0,161834566,250000,3,5187880,1629022144,1,2021-08-15 19:09:04,2021-08-15,260,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000,충청북도,청주시,상당구 용암2동
148,경기도 남양주시 오남읍,조던1 로우 울프그레이 240,1,161646308,240000,3,1278407,1629010574,1,2021-08-15 15:56:14,2021-08-15,240,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000,경기도,남양주시,오남읍
149,경기도 남양주시 다산1동,[235]조던1 로우 울프그레이,2,161695127,225000,3,10281499,1628938834,1,2021-08-14 20:00:34,2021-08-14,235,조던 1 로우 울프그레이,조던,조던 1 로우,울프그레이,119000,경기도,남양주시,다산1동


In [368]:
df.to_csv('./data/jordan1_low_wolfgray.csv')